# net2cog regression tests

This Jupyter notebook runs a suite of regression tests against the net2cog Harmony Service.

These tests use [SMAP_RSS_L3_SSS_SMI_8DAY-RUNNINGMEAN_V4](https://cmr.uat.earthdata.nasa.gov/search/concepts/C1234410736-POCLOUD) as netcdf input data to test the net2cog service for the smap_sss variable.

## Set the Harmony environment:

The cell below sets the `harmony_host_url` to one of the following valid values:

* Production: <https://harmony.earthdata.nasa.gov>
* UAT: <https://harmony.uat.earthdata.nasa.gov>
* SIT: <https://harmony.sit.earthdata.nasa.gov>
* Local: <http://localhost:3000>

The default value is for the UAT environment. When using this notebook there are two ways to use the non-default environment:

* Run this notebook in a local Jupyter notebook server and change the value of `harmony_host_url` in the cell below to the value for the environment you require from the above list.

* Use the `run_notebooks.sh` script, which requires you to declare an environment variable `HARMONY_HOST_URL`. Set that environment variable to the value above that corresponds to the environment you want to test. That environment variable will take precedence over the default value in the cell below.

In [ ]:
harmony_host_url = 'https://harmony.uat.earthdata.nasa.gov'

## Prerequisites

The dependencies for this notebook are listed in the [environment.yaml](./environment.yaml). To test or install locally, create the papermill environment used in the automated regression testing suite:

`conda env create -f ./environment.yaml && conda activate papermill-net2cog`

A `.netrc` file must also be located in the `test` directory of this repository.

### Import required packages:

In [ ]:
from harmony import Collection, Environment, Client, Request

from utility import validate_smap_outputs

### Set up environment dependent variables:

This includes the Harmony `Client` object and `Collection` objects for each of the collections for which there are regression tests. The local, SIT and UAT Harmony instances all utilise resources from CMR UAT, meaning any non-production environment will use the same resources.

When adding a production entry to the dictionary below, the collection instances can be included directly in the production dictionary entry, as they do not need to be shared.

In [ ]:
non_production_collection = {
    'smap_collection': Collection(id='C1272962474-EEDTEST'),
}

non_prod_granule_data = {
    'smap_granules': ['G1272962521-EEDTEST'],
}

collection_data = {
    'https://harmony.uat.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.UAT,
    },
    'https://harmony.sit.earthdata.nasa.gov': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.SIT,
    },
    'http://localhost:3000': {
        **non_production_collection,
        **non_prod_granule_data,
        'env': Environment.LOCAL,
    },
}

environment_information = collection_data.get(harmony_host_url)

if environment_information is not None:
    harmony_client = Client(env=environment_information['env'])
    endpoint_url = environment_information.get('endpoint_url', None)

## Test conversion of sss_smap variable

Use SMAP data.

### Single variable request:

In [ ]:
if environment_information is not None:

    smap_single_variable_request = Request(
        collection=environment_information['smap_collection'],
        granule_id=environment_information['smap_granules'][0],
        variables=['sss_smap'],
        max_results=1,
        format='image/tiff',
    )
    print(harmony_client.request_as_curl(smap_single_variable_request))

    smap_single_variable_job_id = harmony_client.submit(smap_single_variable_request)
    harmony_client.wait_for_processing(smap_single_variable_job_id, show_progress=True)
    validate_smap_outputs(harmony_client, smap_single_variable_job_id, 1)
else:
    print('Skipping test: net2cog is not configured for environment')

### Multiple variables specified:

This test ensures that a request for more than one variable, where those variables are named, will succeed and return output for each variable.

In [ ]:
if environment_information is not None:
    smap_multi_variable_request = Request(
        collection=environment_information['smap_collection'],
        granule_id=environment_information['smap_granules'][0],
        variables=['sss_smap', 'sss_smap_40km'],
        max_results=1,
        format='image/tiff',
    )
    print(harmony_client.request_as_curl(smap_multi_variable_request))

    smap_multi_variable_job_id = harmony_client.submit(smap_multi_variable_request)
    harmony_client.wait_for_processing(smap_multi_variable_job_id, show_progress=True)
    validate_smap_outputs(harmony_client, smap_multi_variable_job_id, 2)
else:
    print('Skipping test: net2cog is not configured for environment')

### No variables specified (so all variables returned):

This should result in 10 output files, 1 each for the following variables:

* `fland`
* `gice`
* `gland`
* `nobs`
* `nobs_40km`
* `sss_ref`
* `sss_smap`
* `sss_smap_40km`
* `sss_smap_uncertainty`
* `surtep`

The input NetCDF-4 file has three additional variables that should not be converted to a GeoTIFF:

* `lat`
* `lon`
* `time`

In [ ]:
if environment_information is not None:

    smap_all_variable_request = Request(
        collection=environment_information['smap_collection'],
        granule_id=environment_information['smap_granules'][0],
        max_results=1,
        format='image/tiff',
    )
    print(harmony_client.request_as_curl(smap_all_variable_request))

    smap_all_variable_job_id = harmony_client.submit(smap_all_variable_request)
    harmony_client.wait_for_processing(smap_all_variable_job_id, show_progress=True)

    # Check all expected variables were produced
    validate_smap_outputs(harmony_client, smap_all_variable_job_id, 10)
else:
    print('Skipping test: net2cog is not configured for environment')